In [1]:
import pandas as pd 
import requests
import unicodedata
from tqdm import tqdm
from bs4 import BeautifulSoup

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_1588/632010323.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
def find_index_of_headers(url):
    response        = requests.get(url)
    content         = response.content
    parser          = BeautifulSoup(content, 'html.parser')
    table           = parser.find('table', {'class': 'infobox'})
    included_headers = {}
    
    for i in range(0, len(table.find_all('th', {'class': 'infobox-label'}))):
        included_headers[unicodedata.normalize('NFKD', table.find_all('th', {'class': 'infobox-label'}).__getitem__(i).getText())] = i
    return included_headers


urls = pd.read_csv('extinct_languages_with_links.csv')['Wikipedia URL'].to_list()
print(len(urls))
print(urls[:10])

urls = list(set(urls))

# urls = [url for url in urls if url not in ['Official_language', 'Minority_language']]
# urls = [url for url in urls if url.startswith('List_of_countries') == False]
# urls = [url for url in urls if url.count(':') == 1]
# urls = [url for url in urls if '?' not in url]

2539
['https://en.wikipedia.org/wiki/Neapolitan_language', 'https://en.wikipedia.org/wiki/Sicilian', 'https://en.wikipedia.org/wiki/Low_Saxon', 'https://en.wikipedia.org/wiki/Belarusian', 'https://en.wikipedia.org/wiki/Lombard', 'https://en.wikipedia.org/wiki/Romani', 'https://en.wikipedia.org/wiki/Yiddish', 'https://en.wikipedia.org/wiki/Gondi', 'https://en.wikipedia.org/wiki/Limburgish_language', 'https://en.wikipedia.org/wiki/Southern_Quechua']


In [7]:
data = pd.read_csv('extinct_languages_with_links.csv')
data['Name in English'].to_list()

['South Italian',
 'Sicilian',
 'Low Saxon',
 'Belarusian',
 'Lombard',
 'Romani',
 'Yiddish (Israel)',
 'Gondi',
 'Limburgian-Ripuarian',
 'Quechua of Southern Bolivia',
 'Kumaoni',
 'Aymara',
 'Emilian-Romagnol',
 'Piedmontese',
 'Venetan',
 'Zazaki',
 'Kurux (India)',
 'Tamajeq',
 'Tulu',
 'Kangdi',
 'Scots',
 'Torlak',
 'West Flemish',
 'Bashkir',
 'Chechen',
 'Chuvash',
 'Meithei',
 'Tamang',
 'Quechua of Cuzco',
 'Eastern Slovak',
 'Ligurian',
 'Rusyn',
 'Talysh',
 'Kui',
 'Khasi',
 'Bodo',
 'Campidanese',
 "K'iche' (Guatemala)",
 'Quechua of Ayacucho',
 'Newar',
 'Mon',
 'Polesian',
 'Tamaceq',
 'Avar',
 'Mundari',
 'Welsh',
 'Angika',
 "Q'eqchi' (Guatemala)",
 'Picard',
 'Kokborok',
 'Basque',
 'Kabard-Cherkes',
 'Mandeali',
 'Friulian',
 'Quechua of Ancash and Huánuco',
 'Walloon',
 'Mising',
 'Ossete',
 'Mizo',
 'Dargwa',
 'Aromanian',
 'Brahui',
 'Languedocian',
 'Lezgian',
 'Mahasui',
 'Magar',
 'Mam (Guatemala)',
 'Udmurt',
 'Huasteca Náhuatl',
 'Kumyk',
 'Yakut',
 'Kaqchi

In [12]:
data = pd.read_csv('extinct_languages_with_links.csv')

data['Wikipedia URL'] = data['Wikipedia URL'].apply(lambda x: x.replace('_people', '_language') if str(x).endswith('_people') else x)
data['Wikipedia URl'] = data['Wikipedia URL'].apply()
data[data['Name in English'] == 'Keiga']



,Name in English,Number of speakers,Degree of endangerment,Wikipedia URL
659,Keiga,6072,Severely endangered,https://en.wikipedia.org/wiki/Keiga_language


In [6]:
data = []
errors = []

for url in tqdm(urls): 
    # try: 
        lang = url.split('/')[-1]

        # print(requests.get(url).status_code)
        # print(url)
        response        = requests.get(url)
        content         = response.content
        parser          = BeautifulSoup(content, 'html.parser')
        table           = parser.find('table', {'class': 'infobox'})

        if table == None:
             print("No table found ", url)
             continue

        header_indices  = find_index_of_headers(url)

        # print(header_indices)

        infobox_data = table.find_all('td', {'class': 'infobox-data'})

        off_lang        = infobox_data[header_indices['Official language in']].text if 'Official language in' in header_indices else None
        rec_min_lang    = infobox_data[header_indices['Recognised minoritylanguage in']].text if 'Recognised minoritylanguage in' in header_indices else None
        # speakers        = infobox_data[header_indices['Speakers']].text             if 'Speakers' in header_indices else None
        iso3code        = infobox_data[header_indices['ISO 639-3']].text            if 'ISO 639-3' in header_indices else None

        if 'Speakers' in header_indices.keys():
            speakers = infobox_data[header_indices['Speakers']].text
        elif 'Native speakers' in header_indices.keys():
            speakers = infobox_data[header_indices['Native speakers']].text
        else:
            speakers = None

        if 'Region' in header_indices.keys():
            regions = infobox_data[header_indices['Region']].text
        elif 'Native Region' in header_indices.keys():
            regions = infobox_data[header_indices['Native Region']].text
        else:
            regions = None

        family          = [x for x in infobox_data[header_indices['Language family']].get_text('title').split('title') if x != "\n"]\
                            if 'Language family' in header_indices else None
        
        dialects        = [x for x in infobox_data[header_indices['Dialects']].get_text('title').split('title') if x != "\n"] \
                            if 'Dialects' in header_indices else None
        
        # print('Regions: ', regions)
        data.append((lang, family, dialects, iso3code, speakers, regions, off_lang, rec_min_lang))
    # except Exception as e: 
    #     errors.append((e, url))
    #     continue

df = pd.DataFrame(data, columns=['lang', 'family', 'dialects', 'iso3code', 'speakers', 'regions', 'off_lang', 'rec_min_lang'])


  0%|          | 4/2266 [00:05<39:04,  1.04s/it]  

No table found  https://en.wikipedia.org/wiki/Guat%C3%B3


  0%|          | 5/2266 [00:05<30:22,  1.24it/s]

No table found  https://en.wikipedia.org/wiki/Andis


  0%|          | 6/2266 [00:05<25:04,  1.50it/s]

...

No table found  https://en.wikipedia.org/wiki/Nyah_Kur


 53%|█████▎    | 1207/2266 [17:44<15:21,  1.15it/s]

No table found  https://en.wikipedia.org/wiki/Southern_Tutchone


 53%|█████▎    | 1208/2266 [17:47<15:35,  1.13it/s]


KeyboardInterrupt: 